In [34]:
ENV["Code"] = "../../Code"
for folder in readdir(ENV["Code"]); push!(LOAD_PATH, normpath(ENV["Code"], folder));end

using CSV
using NoLongerProblems_FileHandling
using NoLongerProblems
using DataFrames
using FQfiles
using HypothesisTests
using MultipleTesting
using Seaborn
using Statistics
using PyPlot
using Random
using PrettyPlotting
using DataFrames

function removeodds(exps)
    exps[!,:TSS1_r2] = [if ii == "NA" 0 else ii end for ii in exps[!,:TSS1_r2]]
    exps[!,:TSS2_r2] = [if ii == "NA" 0 else ii end for ii in exps[!,:TSS2_r2]]
    exps = exps[exps[!,:TSS1_r2].!="\b", :]
    exps = exps[exps[!,:TSS1_r2].!="\x03", :]
    exps = exps[exps[!,:TSS1_r2].!="\x05", :]
    exps = exps[exps[!,:TSS1_r2].!="\x01", :]
    exps = exps[exps[!,:TSS1_r2].!="\x02", :]
    exps = exps[exps[!,:TSS1_r2].!="\f", :]
    exps = exps[exps[!,:TSS2_r2].!="\b", :]
    exps = exps[exps[!,:TSS2_r2].!="\x03", :]
    exps = exps[exps[!,:TSS2_r2].!="\x05", :]
    exps = exps[exps[!,:TSS2_r2].!="\x01", :]
    exps = exps[exps[!,:TSS2_r2].!="\x02", :]
    exps = exps[exps[!,:TSS2_r2].!="\f", :]
end


function sample_from_genotype_timepoint(df)
    df[!,:Sample] = df[!,:Genotype] .*"_".*string.(df[!,:Timepoint])
    df
end

function get_data(genefolder, genename, expnumber; orgainserfolder =  "../TSS_quantification/", addrep = 0)
    expnumber = string(expnumber)
   fol = orgainserfolder * "/"*genefolder*"/" 
      exp = CSV.read(fol* genename*"_exp"*expnumber*"_FQ.csv", DataFrames.DataFrame)
    exp = column_fusion(exp, :Image, :Cell)
    t = CSV.read(genefolder * genename*"_exp"*expnumber*"_TSS.csv", DataFrames.DataFrame)
    t[!,:Image_Cell] = t[!,:Image] .*"__".*string.(t[!,:Cell])
    exp = innerjoin(exp, t, on = :Image_Cell, makeunique=true)
    if addrep > 0
        exp[!,:Rep] = [addrep for ii in 1:nrow(exp)]
    end
    cp = CSV.read(genefolder * genename*"_exp"*expnumber*"_CP.csv", DataFrames.DataFrame)[!,["Genotype","Timepoint","Image_Cell"]]
    cp[!,"Image_Cell"] = [replace(ii, "MAX_" => "") for ii in cp[!,"Image_Cell"]]
    expreturn = sample_from_genotype_timepoint(sort!(innerjoin(exp, cp, on = :Image_Cell, makeunique=true), :Genotype, rev = true))
    
    expreturn[!,:TSS1_r2] = [parse(Float64, split(split(ii, "(")[end], ")")[1]) for ii in  expreturn[!,:TSS1_r2]]
 
    expreturn[!,:TSS2_r2] = [parse(Float64, split(split(ii, "(")[end], ")")[1]) for ii in  expreturn[!,:TSS2_r2]]                   
                            
    removeodds(expreturn)
end
                        
function get_data2(genefolder, genename, expnumber; orgainserfolder =  "../TSS_quantification/", addrep = 0)
    expnumber = string(expnumber)
   fol = orgainserfolder * "/"*genefolder*"/" 
      exp = CSV.read(fol* genename*"_exp"*expnumber*"_FQ.csv", DataFrames.DataFrame)
    exp = column_fusion(exp, :Image, :Cell)
    t = CSV.read(genefolder * genename*"_exp"*expnumber*"_TSS.csv", DataFrames.DataFrame)
    t[!,:Image_Cell] = t[!,:Image] .*"__".*string.(t[!,:Cell])
    exp = innerjoin(exp, t, on = :Image_Cell, makeunique=true)
    if addrep > 0
        exp[!,:Rep] = [addrep for ii in 1:nrow(exp)]
    end
    cp = CSV.read(genefolder * genename*"_exp"*expnumber*"_CP.csv", DataFrames.DataFrame)[!,["Genotype","Timepoint","Image_Cell"]]
    cp[!,"Image_Cell"] = [replace(ii, "MAX_" => "") for ii in cp[!,"Image_Cell"]]
    expreturn = sample_from_genotype_timepoint(sort!(exp, :Genotype, rev = true))
    
    expreturn[!,:TSS1_r2] = [parse(Float64, split(split(ii, "(")[end], ")")[1]) for ii in  expreturn[!,:TSS1_r2]]
 
    expreturn[!,:TSS2_r2] = [parse(Float64, split(split(ii, "(")[end], ")")[1]) for ii in  expreturn[!,:TSS2_r2]]                   
                            
    removeodds(expreturn)
end

if !in("GeneData", readdir())
    mkdir("GeneData")
end

if !in("CompleteSets", readdir())
    mkdir("CompleteSets")
end

include("nascentmaturetimecourses_genedata.jl")
include("Il12bHSS1_timecourse_genedata.jl")
include("mixingexperiments.jl")
include("IFNGrescue.jl")
include("enhancersets.jl")
include("lpsrepressedgenes.jl")
include("BD_inhibitors.jl")
include("MakeCompleteSetsFolder.jl")

import Pkg; Pkg.status()

      Status `~/.julia/environments/v1.6/Project.toml`
  [336ed68f] CSV v0.9.11
  [a93c6f00] DataFrames v0.22.7
  [31c24e10] Distributions v0.23.12
  [c2308a5c] FASTX v1.2.0
  [53c48c17] FixedPointNumbers v0.8.4
  [09f84164] HypothesisTests v0.10.2
  [7073ff75] IJulia v1.23.2
  [6a3955dd] ImageFiltering v0.6.21
  [6218d12a] ImageMagick v1.2.2
  [86fae568] ImageView v0.10.13
  [916415d5] Images v0.23.3
  [4138dd39] JLD v0.12.3
  [093fc24a] LightGraphs v1.3.5
  [10e44e05] MATLAB v0.8.0
  [f8716d33] MultipleTesting v0.4.1
  [eadc2687] Pandas v1.4.0
  [91a5bcdd] Plots v1.21.2
  [92933f4c] ProgressMeter v1.5.0
  [438e738f] PyCall v1.92.2
  [d330b81b] PyPlot v2.9.0
  [6f49c342] RCall v0.13.10
  [3646fa90] ScikitLearn v0.6.3
  [d2ef9438] Seaborn v0.4.1
  [2913bbd2] StatsBase v0.33.10
